In [1]:
import pandas as pd
from extract import CategorizeUsers
from rich import print
import random
from pathlib import Path
import json

# set paths
my_saving_dir = Path.cwd() / "categorizing_dicts"

In [2]:
df = pd.read_csv("df_nodes.csv")

In [3]:
df = df.drop(columns=df.columns[0])

In [4]:
df.head()

,user_id,tweet_id,retweet_id,hashtag,location,verified
0,1007449601400541184,1503521257177948170,278369814,"['putin', 'slavaukraini']","Mobile, AL",False
1,1464582321533767695,1503521249850363904,291496884,"['stopputin', 'stopwar', 'stopwarinukraine']",NaN,False
2,1245946724994158592,1503521066739589120,3312966191,"['standwithukraine', 'stopputinnow', 'slavaukr...",NaN,False
3,854862661,1503521023450361861,465973,['slavaukraini'],NaN,False
4,42746752,1503520988297932802,1456400208431091712,"['dogsoftwittter', 'dogsoftwitter', 'stoprussi...","Northeast Ohio, USA",False


In [5]:
unique_users = set(df["user_id"])
assert len(unique_users) == df["user_id"].nunique()
print(f"We have {len(unique_users)} unique users.")

We have 32382 unique users.

Dividing the set into `n_sets` ~equal parts so that I have total 20 jsons

In [6]:
n_sets = [x for x in my_saving_dir.glob('*.json')]
n_sets = 20 - len(n_sets)

Remove all users already categorized

In [7]:
# load all jsons
users_already_done = set()
for json_file in my_saving_dir.glob('*.json'):
    with open(json_file) as f:
        data = json.load(f)
    data = {int(k):v for k,v in data.items()}
    users_already_done.update(data.keys())


# remove users from `unique_users`
print(f"The total number of users is {len(unique_users)}, we've already done {len(users_already_done)}")
unique_users -= users_already_done
print(f"and so left to do are {len(unique_users)} users.")

The total number of users is 32382, we've already done 29142

and so left to do are 3240 users.

In [8]:
slen = len(unique_users) // n_sets
print(f"Length of each set: {slen}")
unique_users_list = []
while len(unique_users) >= slen and slen>0:
    my_set = set(random.sample(unique_users, slen))
    unique_users_list.append(my_set)
    unique_users -= my_set

# need to add any leftover in `unique_users:set`
unique_users_list[-1] = unique_users_list[-1] | unique_users

# print length of each set
my_repr = "Lengths of each set:"
for my_set in unique_users_list:
    my_repr = f"{my_repr} {len(my_set)},"
print(f"{my_repr[:-1]}. We have [red]{len(unique_users_list)}[/red] sets.")

Length of each set: 1620

/var/folders/hb/d5v787851zs4pcg7b14rpb9r0000gn/T/ipykernel_858/2975302905.py:5: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  my_set = set(random.sample(unique_users, slen))


Lengths of each set: 1620, 1620. We have 2 sets.

In [9]:
for i, my_set in enumerate(unique_users_list):
    c = CategorizeUsers(my_set, pages=1)
    with open(my_saving_dir / f"hashtags_{i}_no_overwrite.json", "w", encoding="utf-8") as f:
        json.dump(c.users, f, ensure_ascii=False, indent=4)

Output()

Output()